In [7]:
import csv
import pandas as pd
from pathlib import Path
import os

import json
import time
#filter datasets into one file and then seperate into weather station with all years and times in one
weatherStations = [6092, 6098, 6121, 6124, 6127, 6129,6131, 6132, 6133, 6136, 6143, 6003, 6029, 6026, 6062, 6029, 6062, 6087]
#weatherStations = map(int, os.environ.get("WEATHER_STATIONS").split(", "))
#FLOOD_DATA_DIR = os.environ.get("FLOOD_DATA_DIR")


def newFillOutData():
    #first do pathlib to get all the files
    directory = "./RFbyStation"
    files = list(Path(directory).glob('*'))
    for oldFile in files:
        if str(oldFile) != "RFbyStation\\6993_2011-2023_60rf.txt":
            df = pd.read_csv(oldFile)

            df['obstime'] = pd.to_datetime(df['obstime'], format="%Y-%m-%dT%H:%M:%S")
            #If you want to fill out the data for all years just change the start to a gen. variable
            # of the date time format in the df, this is just grabbing the first date time and the last date time
            # but ive now changed it to generic start, so if ya want change it to df['obstime'].iloc[1]
            #change end to last date of last year, but thatll blow the data significantly up - I recomend doing 
            #this after figuring out which weather stations we want coords wise
            start = df['obstime'].iloc[1]
            print(start)
            end = df['obstime'].iloc[-1]
            dates = pd.date_range(start = start, end=end, freq='1h')
            #we reindex the dataframe with the date range created earlier
            #https://stackoverflow.com/questions/49187686/how-to-fill-missing-timestamps-for-time-column-for-a-date-in-pandas
            df = df.set_index('obstime').reindex(dates).reset_index()

            #https://stackoverflow.com/questions/38134012/pandas-dataframe-fillna-only-some-columns-in-place
            #s for synthethised, we pad everything that was NaN values
            filename = str(oldFile).split("\\")[-1]
            stationid = filename.split("_")[0]
            paddedDetails = {'station_id':stationid, 'Past 60-Minutes Rainfall in mm':0, 'qcscore':'S'}
            df = df.fillna(paddedDetails)
            df['station_id'] = df['station_id'].astype(int)

            newFileName = Path(oldFile).stem +"_padded.csv"
            #had a few issues with reset index - using this instead is simpler and quicker to rename the columns
            df = df.rename(columns={"obstime": "index","index":"obstime", "Past 60-Minutes Rainfall in mm": "60rf_in_mm", "qcscore": "qcscore"})
            df.to_csv("./preprocessedRFData/"+newFileName, encoding='utf-8', index=False)
        
            print(newFileName, " Completed")
        else:
            print(oldFile, "Ignored")
    print("Finished padding data.")

def RHFillOutData():
    #first do pathlib to get all the files
    directory = "./RHbyStation"
    files = list(Path(directory).glob('*'))
    for oldFile in files:
            df = pd.read_csv(oldFile)
            df['obstime'] = pd.to_datetime(df['obstime'], format="%Y-%m-%dT%H:%M:%S")
            
            start = df['obstime'].iloc[1]
            print(start)
            print(df['obstime'].iloc[0])
            end = df['obstime'].iloc[-1]
            dates = pd.date_range(start = start, end=end, freq='1h')
            #we reindex the dataframe with the date range created earlier
            #https://stackoverflow.com/questions/49187686/how-to-fill-missing-timestamps-for-time-column-for-a-date-in-pandas
            df = df.set_index('obstime').reindex(dates).reset_index()

            #https://stackoverflow.com/questions/38134012/pandas-dataframe-fillna-only-some-columns-in-place
            #s for synthethised, we pad everything that was NaN values
            filename = str(oldFile).split("\\")[-1]
            stationid = filename.split("_")[0]
            paddedDetails = {'station_id':stationid, "Relative Humidity in %":0, 'qcscore':'X'}
            df = df.fillna(paddedDetails)

            newFileName = Path(oldFile).stem +"_padded.csv"
            #had a few issues with reset index - using this instead is simpler and quicker to rename the columns
            df = df.rename(columns={"obstime": "index","index":"obstime", "Relative Humidity in %": "Relative Humidity in %", "qcscore": "qcscore"})
            df['station_id'] = df['station_id'].astype(int)
            df.to_csv("./preprocessedRHData/"+newFileName, encoding='utf-8', index=False)
        
            print(newFileName, " Completed")

    print("Finished padding data.")
  
def filterIntoStations(uniqueStations, datatype, directory, uniqueColumn, fileEnd):
    
    files = list(Path(directory).glob('*'))
    print("Started filtering data grouped by station id")
    for station in uniqueStations: 
        newCSV = [['obstime,station_id,'+uniqueColumn+',qcscore']]
        for file in files:
            with open(file, 'r') as file:
                reader = csv.reader(file, delimiter=',', quotechar= '|')
                for row in reader:
                    if row[1] == station:
                        newCSV.append([row[0], row[1], row[2], row[3]])
        newFilename = "./"+datatype+"/"+station+"_2011-2023_60"+fileEnd+".txt"
        with open(newFilename, 'w') as newFile:
                for row in newCSV:
                    formatted_row = ','.join(row) + '\n'
                    newFile.write(formatted_row)
                print(newFilename, " Completed")
    print("Finished filtering data grouped by station id")


def joinRHRF():
    rhDirectory = "./collatedRHPadded"
    rfDirectory = "./collatedRFPadded"
    
    RHFiles = list(Path(rhDirectory).glob('*'))
    RFFiles = list(Path(rfDirectory).glob('*'))

    years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    #https://stackoverflow.com/questions/41815079/pandas-merge-join-two-data-frames-on-multiple-columns
    for i in range(len(years)):
        RHFile = pd.read_csv(RHFiles[i])
        RFFile = pd.read_csv(RFFiles[i])
        combinedDF = pd.merge(RFFile, RHFile, how='left', left_on=['obstime', 'station_id'], right_on=['obstime', 'station_id'])
        ###Finally pad the data again
        paddedDetails = {'Past 60-Minutes Rainfall in mm':0,'qcscore_x':'S',"Relative Humidity in %":0, 'qcscore_y':'X'}
        combinedDF = combinedDF.fillna(paddedDetails)  
        ###Somewhere in the code it converts station id to float, so we're reverting it here just as a quick fix     
        combinedDF['station_id'] = combinedDF['station_id'].astype(int)
        combinedDF.to_csv("./combinedPreprocessed/"+str(years[i]), encoding='utf-8', index=False)
        print("Completed combining RF and RH for: " +str(years[i]))

def addHumidity():
    uniqueStations = getStationNames("./collatedRH")
    print("Found these unique weather RH stations: \n", uniqueStations)

    uniqueStations2 = getStationNames("./collatedRF")
    print("Found these unique weather RH stations: \n", uniqueStations2)
    matchupData = []
    mismatchData = []
    for uniqueStation in uniqueStations:
        if uniqueStation not in uniqueStations2:
            mismatchData.append(uniqueStation)
        if uniqueStation in uniqueStations2:
            matchupData.append(uniqueStation)
    print("\n MatchUps: \n")
    print(matchupData)
    print("\n Mismatchs: \n")
    print(mismatchData)

    ###Need to split RH into by weather station
    filterIntoStations(uniqueStations,"RHbyStation", "./collatedRH", "Relative Humidity in %", "rh")
    ###Then need too pad and save the data
    RHFillOutData()
    FillOutDataYear("RH")

    ###Then need to conditionally check where the data matches up and link it
    joinRHRF()

def getStationCoords(directory):
    files = Path(directory).glob('*')
    station_coords = {}
    for file in files:
        df = pd.read_csv(file)
        stationData = df[['station_id', 'latitude', 'longitude']]
        for index, row in stationData.iterrows():
            station_id = row['station_id']
            latitude = row['latitude']
            longitude = row['longitude']
            if station_id not in station_coords:
                station_coords[station_id] = [longitude, latitude]

    with open("station_coords.json", 'w') as json_file:
        json.dump(station_coords, json_file)
    print("Finished getting stations")



def dropQScores():
    rhDirectory = "./collatedRHPadded"
    rfDirectory = "./collatedRFPadded"
    combinedDirectory = "./combinedPreprocessed"


    RHFiles = list(Path(rhDirectory).glob('*'))
    RFFiles = list(Path(rfDirectory).glob('*'))
    combinedFiles = list(Path(combinedDirectory).glob('*'))

    #for file in RHFiles:
    #    df = pd.read_csv(file)
    #    df.drop(columns=['qcscore'], inplace=True)
    #    df.to_csv(file, encoding='utf-8', index=False)
#
    #for file in RFFiles:
    #    df = pd.read_csv(file)
    #    df.drop(columns=['qcscore'], inplace=True)
    #    df.to_csv(file, encoding='utf-8', index=False)
    
    for file in combinedFiles:
        df = pd.read_csv(file)
        df.drop(columns=['qcscore_x', 'qcscore_y'], inplace=True)
        df.to_csv(file, encoding='utf-8', index=False)

#https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
def getStationNames():
    files = Path("./collatedRFPadded").glob('*')
    uniqueStations = []
    for file in files:
        print(file)
        with open(file, 'r') as file:
            reader = csv.reader(file, delimiter=',', quotechar='|')
            for row in reader:
                station = row[1]
                if row[1] not in uniqueStations:
                    uniqueStations.append(row[1])
        #get rid of station_id header
    del uniqueStations[0]
    print("Finished getting unique station names")
    return uniqueStations

def FillOutDataYear(datatype, year):
    #first do pathlib to get all the files
    #Issue is that 
    #years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    
    directory = "./preprocessed"+datatype+"Data"
    files = list(Path(directory).glob('*'))
    finalYearDF = pd.DataFrame()
    #Directory is by weather station, so need to iterate through each file and where the year 
    for oldFile in files:
        #6993 is just one entry long so we ignore
        #https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
        df = pd.read_csv(oldFile)
            
        #just in case this isnt datetime
        #df['obstime'] = pd.to_datetime(df['obstime'], format="%Y-%m-%d %H:%M:%S")
        
        df['obstime'] = pd.to_datetime(df['obstime'], format="ISO8601")
        #df['obstime'] = addT(df['obstime'])
        #create year column
        df['year'] = df['obstime'].dt.year
        #dataframe that includes just the year
        
        yearDF = df[df['year'] == year]

        
        finalYearDF = pd.concat([finalYearDF, yearDF], ignore_index=True)
        #print(Path(oldFile).stem, " Completed")
        finalYearDF.drop(columns=['year'], inplace=True)
        newFileName = str(year)+ "_Year_"+datatype+"_padded.csv"
        finalYearDF['station_id'] = finalYearDF['station_id'].astype(int)
        #print("Completed: ", oldFile)
    finalYearDF.to_csv("./collated"+datatype+"Padded/"+newFileName, encoding='utf-8', index=False)
    print("Finished padding data.")

def fixTime():
    dataDir = './collatedRFPadded'
    files = list(Path(dataDir).glob('*'))
    #apply the removepunctuation function to the text column
    for file in files:
        df = pd.read_csv(file)
        df['obstime'] = df['obstime'].apply(addT)
        display(df)
        df.to_csv(str(file), encoding='utf-8', index=False)

        


def addT(obstime): 
    #loop through all punctuation marks and replace them with empty string
    obstime = obstime.replace(" ", "T")
    return obstime

#uniqueStations = getStationNames()
#print("Found these unique weather stations: \n", uniqueStations)
#filterIntoStations(uniqueStations, "RFbyStation","./collatedRF", "Past 60-Minutes Rainfall in mm", "rf")
#newFillOutData()
#FillOutDataYear("RF", 2023)
#PuthearaKeFunction()
# addHumidity()

##Only run this if you want to drop the q scores
# dropQScores()

#fixTime()

test1 = ['1005', '1010', '1012', '1013', '1016', '1017', '6001', '6002', '6003', '6007', '6009', '6015', '6016', '6017', '6018', '6025', '6028', '6029', '6031', '6034', '6040', '6041', '6042', '6043', '6045', '6047', '6049', '6053', '6055', '6056', '6058', '6061', '6021', '6027', '6020', '6036', '6051', '6013', '6039', '6037', '6026', '6046', '6101', '6048', '6998', '6060', '6063', '6054', '6062', '6059', '6065', '6993', '6064', '6999', '6067', '6068', '6066', '6070', '6069', '6050', '6071', '6072', '6077', '6075', '6074', '6076', '6079', '6080', '6081', '6102', '6103', '6082', '6078', '6084', '6083', '6085', '6011', '6086', '6087', '6088', '6089', '6092', '6104', '6093', '6094', '6095', '6090', '6097', '6096', '6091', '6100', '6106', '6098', '6107', '6108', '6111', '6995', '6109', '6112', '6116', '6120', '6117', '6121', '6115', '6119', '6123', '6124', '6005', '6125', '6126', '9913', '6127', '6128', '6129', '6130', '6133', '6131', '6132', '6134', '6135', '6136', '6110', '6138', '6139', '6144', '6143', '6142', '6147', '6149']
test2 =  ['1005', '1010', '1012', '1013', '1016', '1017', '6001', '6002', '6003', '6007', '6009', '6013', '6015', '6016', '6017', '6018', '6020', '6021', '6025', '6026', '6027', '6028', '6029', '6031', '6034', '6036', '6037', '6039', '6040', '6041', '6042', '6043', '6045', '6046', '6047', '6048', '6049', '6051', '6053', '6054', '6055', '6056', '6058', '6059', '6060', '6061', '6062', '6063', '6064', '6065', '6066', '6067', '6068', '6069', '6070', '6101', '6998', '6999', '6050', '6071', '6072', '6074', '6075', '6076', '6077', '6079', '6080', '6081', '6102', '6103', '6078', '6082', '6083', '6084', '6011', '6085', '6086', '6087', '6088', '6089', '6092', '6090', '6091', '6093', '6094', '6095', '6096', '6097', '6104', '6098', '6100', '6106', '6107', '6108', '6111', '6995', '6109', '6112', '6116', '6115', '6117', '6119', '6120', '6121', '6123', '6124', '6005', '6125', '6126', '6127', '6128', '9913', '6129', '6130', '6131', '6132', '6133', '6134', '6110', '6135', '6136', '6138', '6139', '6142', '6143', '6144', '6147', '6149']

for station in test2:
    if station not in test1:
        print(station)